In [ ]:
"""
Read a social graph and preprocess it in such a way that it can be compressed
using the WebGraph method.
"""
# NOTE: unsure if we can improve memory management by freeing memory or if
# python does this by itself.

import requests  # read social graph from web
import gzip
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import zipfile
import io
import pandas as pd
from collections import deque


# -1: none, 0: size, 1: modularity contribution, 2: edge density, 3: edge count
#COMM_SORTING = 0
# -1: none, 0: outdegree, 1: indegree, 2: betweenness, 3: pagerank, 4: closeness
#NODE_SORTING = 0
# -1: none, 0: BFS, 1: DFS
#SEARCH = -1


# ====== COMMUNITY SORTING METHODS ======
def community_sort_by_size(communities, subgraphs):
    sorted_communities = sorted(subgraphs, key=len, reverse=True)


    print(f"Found {communities} communities with largest community of"
          f" size {len(sorted_communities[0])} and smallest community of size "
          f" {len(sorted_communities[-1])}")
    return sorted_communities


def community_sort_by_modularity_contribution(communities, subgraphs):
    sorted_c = sorted(subgraphs, key=lambda x:
                      nx.community.modularity(x, [x.nodes()]),
                      reverse=True)

    print(f"Found {communities} communities with highest modularity"
          f" contribution "
          f" {nx.community.modularity(sorted_c[0], [sorted_c[0].nodes()])} and"
          f" smallest modularity contribution "
          f" {nx.community.modularity(sorted_c[-1], [sorted_c[-1].nodes()])}")
    return sorted_c


def community_sort_by_edge_density(communities, subgraphs):
    sorted_communities = sorted(subgraphs, key=lambda x: nx.density(x),
                                reverse=True)
    print(f"Found {communities} communities with highest density"
          f" {nx.density(sorted_communities[1])} and smallest density"
          f" {nx.density(sorted_communities[-1])}")
    return sorted_communities


def community_sort_by_edge_count(communities, subgraphs):
    sorted_communities = sorted(subgraphs, key=lambda x: x.number_of_edges(),
                                reverse=True)
    print(f"Found {communities} communities with highest edge count"
          f" {nx.density(sorted_communities[1])} and smallest edge count"
          f" {nx.density(sorted_communities[-1])}")
    return sorted_communities


def community_sort_by_nothing(communities, subgraphs):
    return subgraphs


community_sort = [
    community_sort_by_size,
    community_sort_by_modularity_contribution,
    community_sort_by_edge_density,
    community_sort_by_edge_count,
    community_sort_by_nothing
]


# ====== NODE SORTING METHODS ======
def node_sort_by_outdegree(C):
    sorted_nodes = sorted(C.nodes, key=lambda x: C.out_degree(x), reverse=True)
    return sorted_nodes


def node_sort_by_indegree(C):
    sorted_nodes = sorted(C.nodes, key=lambda x: C.in_degree(x), reverse=True)
    return sorted_nodes



def node_sort_by_nothing(C):
    return list(C.nodes)


node_sort = [
    node_sort_by_outdegree,
    node_sort_by_indegree,
    node_sort_by_nothing
]


# ====== NODE RELABELLING ======
def bfs(C, sorted_nodes):
    visited = set()
    bfs_order = []

    #deque to efficiently pop first element O(1) compared to O(n) for a normal list
    queue = deque()

    #precompute indices
    node_index = {node: i for i, node in enumerate(sorted_nodes)}

    # make sure all nodes are visited
    for node in sorted_nodes:
        if node not in visited:
            queue.append(node)
        while queue:
        # sort successors based on sorted_nodes
          current = queue.popleft()
          if current not in visited:
              visited.add(current)
              bfs_order.append(current)
              nb = sorted(C.successors(current),key=lambda x: node_index[x])
              queue.extend(nb)


    return bfs_order


def dfs(C, sorted_nodes):
    visited = set()
    dfs_order = []
    stack = []

    #precompute indices
    node_index = {node: i for i, node in enumerate(sorted_nodes)}

    # make sure all nodes are visited
    for node in sorted_nodes:
        if node not in visited:
            stack.append(node)
            while stack:
                current = stack.pop()
                if current not in visited:
                    visited.add(current)
                    dfs_order.append(current)
                    nb = sorted(C.successors(current),key=lambda x: node_index[x])
                    stack.extend(nb)
    return dfs_order

def no_order(C,sorted_nodes):
  return sorted_nodes

order_nodes = [bfs, dfs,no_order]


# ====== GAP DISTRIBUTION ======
def compute_gap_distribution(G):
    gap_distribution = Counter()

    for node in G.nodes:
        nb = sorted(G.successors(node))
        if len(nb) > 1:
            gaps = [nb[i] - nb[i - 1] for i in range(1, len(nb))]
            gap_distribution.update(gaps)
    return gap_distribution


def plot_gap_distribution(gap_distribution,graph_name,c,n,s):
    gap_sizes = np.array(list(gap_distribution.keys()))
    frequencies = np.array(list(gap_distribution.values()))
    probabilities = frequencies / np.sum(frequencies)
    sorted_indices = np.argsort(gap_sizes)
    gap_sizes = gap_sizes[sorted_indices]
    probabilities = probabilities[sorted_indices]

    plt.figure()
    plt.loglog(gap_sizes, probabilities, 'o', color='black',
               markersize=3, alpha=0.7, label='Gap distribution')
    plt.xlabel('Gap Size (log scale)')
    plt.ylabel('Probability Density (log scale)')
    plt.title('gap distribution')
    plt.savefig(f"{graph_name}{c}{n}{s}.png")
    plt.close()


#Export to adjacency list .graph-txt
def export(G,graph_name,c,n,s):
    with open(f"{graph_name}{c}{n}{s}.graph-txt", "w") as f:
        f.write(f'{G.number_of_nodes()}\n')
        for i in nx.generate_adjlist(G):
            f.write(" ".join(map(str, sorted(map(int, i.split()[1:])))))
            f.write('\n')


# ====== MAIN ======
# URL of zipped source data file
#urls = ["https://snap.stanford.edu/data/soc-Slashdot0902.txt.gz","https://networks.skewed.de/net/prosper/files/prosper.csv.zip","https://networks.skewed.de/net/flickr_aminer/files/flickr_aminer.csv.zip"]
#url = "https://snap.stanford.edu/data/soc-Slashdot0902.txt.gz"
#url = "https://networks.skewed.de/net/flickr_aminer/files/flickr_aminer.csv.zip"
#url = "https://networks.skewed.de/net/prosper/files/prosper.csv.zip"

#Function to generate the different orderings
def main_generation(url,graph_name):
  ok = False
  if url.endswith(".gz"):
    # fetch file
    # NOTE: stream=True reads the file in chunks
    response = requests.get(url, stream=True)


    if response.status_code == 200:
        # load graph
        ok = True
        G = nx.DiGraph()

        # decompress data
        with gzip.GzipFile(fileobj=response.raw) as gz:
            for line in gz.read().decode('utf-8').splitlines():
                if line.startswith('#'):
                    continue
                source, target = map(int, line.split())
                G.add_edge(source, target)


  elif url.endswith(".zip"): #files from https://networks.skewed.de/ zip with 3 files, we want edges.csv
      response = requests.get(url, stream=True)
      if response.status_code == 200:
        # load graph
        ok = True

        # decompress data
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_file:
          # Check if edges.csv is in the zip
          if "edges.csv" not in zip_file.namelist():
              raise ValueError(f"File edges.csv not found in the ZIP archive.")

          # Open the CSV file directly and load it into a DataFrame
          with zip_file.open("edges.csv") as csv:
            df = pd.read_csv(csv,skiprows=1,header=None,usecols=[0,1])
            G = nx.from_pandas_edgelist(df,0,1,create_using=nx.DiGraph())

  print(f"Graph loaded with {G.number_of_nodes()} nodes "
        f"and {G.number_of_edges()} edges")
  if ok:
      # community processing
      communities = nx.community.louvain_communities(G)
      communities_len = len(communities)
      subgraphs = []
      while communities:
      # Delete communities dinamically
        community = communities.pop(0)
        subgraphs.append(G.subgraph(community))

      #free some ram
      del communities

      for comm_s in range(-1,4):
        sorted_communities = community_sort[comm_s](communities_len,subgraphs)

        for node_s in range(-1,2):
          for search in range(-1,2):
            # node processing
            relabel_map = {}
            new_label = 0
            for C in sorted_communities:
                sorted_nodes = node_sort[node_s](C)
                order = order_nodes[search](C, sorted_nodes)
                # relabel nodes
                for node in order:
                    relabel_map[node] = new_label
                    new_label += 1

            # relabel graph
            G_relabelled = nx.relabel_nodes(G, relabel_map)

            # plot gap distr
            gap_distr = compute_gap_distribution(G_relabelled)
            plot_gap_distribution(gap_distr,graph_name,comm_s,node_s,search)

            # store new graph as adj list
            export(G_relabelled,graph_name,comm_s,node_s,search)
  else:
      print(f"Failed to fetch file. Status Code: {response.status_code}")


def main():
   urls = ["https://snap.stanford.edu/data/soc-Slashdot0902.txt.gz","https://networks.skewed.de/net/prosper/files/prosper.csv.zip","https://networks.skewed.de/net/flickr_aminer/files/flickr_aminer.csv.zip"]
   names = ['slash','prosper','flickr']
   for url,name in zip(urls,names):
      main_generation(url,name)

main()

Graph loaded with 82168 nodes and 948464 edges
Found 1014 communities with largest community of size 18263 and smallest community of size  2
Found 1014 communities with highest modularity contribution  1.1102230246251565e-16 and smallest modularity contribution  0.0
Found 1014 communities with highest density 2.0 and smallest density 0.0005258412547083613
Found 1014 communities with highest edge count 0.0010332122240658877 and smallest edge count 1.0
Graph loaded with 89269 nodes and 3330225 edges
Found 51 communities with largest community of size 27112 and smallest community of size  2
Found 51 communities with highest modularity contribution  1.1102230246251565e-16 and smallest modularity contribution  0.0
Found 51 communities with highest density 0.5 and smallest density 0.0007468748714004073
Found 51 communities with highest edge count 0.0007468748714004073 and smallest edge count 0.5
Graph loaded with 214626 nodes and 9114557 edges
Found 172 communities with largest community of 

In [ ]:
import os
def create_zip_name(name):
  zip_file = zipfile.ZipFile(f'session_files_{name}.zip', 'w')
  for root, dirs, files in os.walk('/content'):
      for file in files:
        if(file.startswith(f'{name}')):
          zip_file.write(os.path.join(root, file))
  zip_file.close()

create_zip_name('prosper')

In [ ]:
import os
def create_zips_all_names():
  names = ['slash','flickr','prosper']
  for name in names:
    zip_file = zipfile.ZipFile(f'session_files_{name}.zip', 'w')
    for root, dirs, files in os.walk('/content'):
        for file in files:
          if(file.startswith(f'{name}')):
            zip_file.write(os.path.join(root, file))
    zip_file.close()

create_zips_all_names()

## Ignore spaghetti below here

In [ ]:
import pandas as pd
import random
import networkx as nx
import numpy as np
import math
import matplotlib.pyplot as plt


In [ ]:
G = nx.read_edgelist("Slashdot0902.txt",create_using=nx.DiGraph)
print(f'Edges: {G.number_of_edges()}')
print(f'Nodes: {G.number_of_nodes()}')

Edges: 948464
Nodes: 82168


In [ ]:
total_edges = G.number_of_edges()

# Sort communities by size

In [ ]:
communities = nx.community.louvain_communities(G)

In [ ]:
sorted_communities_size = sorted(communities, key=len, reverse=True)

In [ ]:
G_communities_size = nx.DiGraph()

for community in sorted_communities_size:
    for node in community:
        G_communities_size.add_node(node)

G_communities_size.add_edges_from(G.edges)

In [ ]:
def export_adj(G,file):
  with open(f"{file}.graph-txt", "w") as f:
    f.write(str(G.number_of_nodes())+'\n')
    for i in nx.generate_adjlist(G):
      f.write(" ".join(map(str, sorted(map(int, i.split()[1:])))))
      f.write('\n')
  print(f'{file}.graph-txt created')

In [ ]:
export_adj(G_communities_size,'communities_size')

communities_size.graph-txt created


# Sort communities by modularity contribution and nodes within communities by outdegree

In [ ]:
def modularity_contribution(G,community,total_edges):
    edges_inside = community.number_of_edges()
    degree_community_sum = sum(G.degree(n) for n in community.nodes)
    return edges_inside / total_edges - (degree_community_sum / (2 * total_edges)) ** 2

In [ ]:
communities_modularity = nx.community.modularity(G, communities)
communities_modularity

0.3965203557855198

In [ ]:
def sort_communities_mod(G,communities):
  community_modularities = []
  for community in communities:
      community_modularities.append(modularity_contribution(G,G.subgraph(community),total_edges))


  communities_modularity_zip = list(zip(communities,community_modularities))
  sorted_communities_modularity = sorted(communities_modularity_zip,key=lambda x: x[1],reverse=True)
  sorted_communities_modularity = [c for c,m in sorted_communities_modularity]
  return sorted_communities_modularity

In [ ]:
community_modularities = []
for community in communities:
    community_modularities.append(modularity_contribution(G.subgraph(community),total_edges))


communities_modularity_zip = list(zip(communities,community_modularities))
sorted_communities_modularity = sorted(communities_modularity_zip,key=lambda x: x[1],reverse=True)
sorted_communities_modularity = [c for c,m in sorted_communities_modularity]


In [ ]:
sum(community_modularities)

0.3965181224153244

In [ ]:
def sort_nodes_degree(G,communities_ordered):
  sorted_nodes = []
  for c in communities_ordered:
      for n in sorted(c, key=lambda node: G.out_degree(node), reverse=True):
        sorted_nodes.append(n)
  return sorted_nodes


In [ ]:
sorted_nodes_mod_degree = sort_nodes_degree(sorted_communities_modularity)


In [ ]:
G_mod_deg = nx.DiGraph()
G_mod_deg.add_nodes_from(sorted_nodes_mod_degree)
G_mod_deg.add_edges_from(G.edges)


In [ ]:
export_adj(G_mod_deg,"mod_outdegree")

mod_outdegree.graph-txt created


# Sort communities by clustering coefficient

In [ ]:
clustering_coeffs = []
for community in communities:
  subgraph = G.subgraph(community)
  clustering_coeffs.append(nx.average_clustering(subgraph))


In [ ]:
communities_modularity_zip = list(zip(communities,clustering_coeffs))
sorted_communities_clustering = sorted(communities_modularity_zip,key=lambda x: x[1],reverse=True)
sorted_communities_clustering = [c for c,m in sorted_communities_clustering]

In [ ]:
G_clustering = nx.DiGraph()

for community in sorted_communities_clustering:
    for node in community:
        G_clustering.add_node(node)
G_clustering.add_edges_from(G.edges)


In [ ]:
export_adj(G_clustering,"clustering")

clustering.graph-txt created


In [ ]:
import zipfile
import os

In [ ]:
file_path = '/content/network.csv.zip'
try:
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        extract_path = '/content/data'
        os.makedirs(extract_path, exist_ok=True)
        zip_ref.extractall(extract_path)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except zipfile.BadZipFile:
    print(f"Error: Invalid zip file at {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
df = pd.read_csv('data/edges.csv',skiprows=1,header=None,usecols=[0,1])
G_prosper = nx.from_pandas_edgelist(df,0,1,create_using=nx.DiGraph())

In [ ]:
print(f'Edges: {G_prosper.number_of_edges()}')
print(f'Nodes: {G_prosper.number_of_nodes()}')

Edges: 3330225
Nodes: 89269


In [ ]:
total_edges = G_prosper.number_of_edges()

In [ ]:
communities = nx.community.louvain_communities(G_prosper)

In [ ]:
sorted_communities_size = sorted(communities, key=len, reverse=True)

In [ ]:
G_prosper_communities_size = nx.DiGraph()

for community in sorted_communities_size:
    for node in community:
        G_prosper_communities_size.add_node(node)

G_prosper_communities_size.add_edges_from(G_prosper.edges)

In [ ]:
export_adj(G_prosper_communities_size,'prosper_communities_size')

prosper_communities_size.graph-txt created


In [ ]:
sorted_communities_modularity = sort_communities_mod(G_prosper,communities)

In [ ]:
G_prosper_communities_mod = nx.DiGraph()

for community in sorted_communities_size:
    for node in community:
        G_prosper_communities_mod.add_node(node)

G_prosper_communities_mod.add_edges_from(G_prosper.edges)

In [ ]:
export_adj(G_prosper_communities_size,'prosper_communities_mod')

prosper_communities_mod.graph-txt created


In [ ]:
sorted_nodes_mod_degree = sort_nodes_degree(G_prosper,sorted_communities_modularity)

In [ ]:
G_prosper_mod_deg = nx.DiGraph()
G_prosper_mod_deg.add_nodes_from(sorted_nodes_mod_degree)
G_prosper_mod_deg.add_edges_from(G_prosper.edges)

In [ ]:
export_adj(G_prosper_mod_deg,'prosper_mod_deg')

prosper_mod_deg.graph-txt created


In [ ]:
file_path = '/content/network.csvflicker.zip'
try:
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        extract_path = '/content/data'
        os.makedirs(extract_path, exist_ok=True)
        zip_ref.extractall(extract_path)
except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except zipfile.BadZipFile:
    print(f"Error: Invalid zip file at {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
df = pd.read_csv('data/edges.csv',skiprows=1,header=None,usecols=[0,1])
G_flicker = nx.from_pandas_edgelist(df,0,1,create_using=nx.DiGraph())

In [ ]:
print(f'Edges: {G_flicker.number_of_edges()}')
print(f'Nodes: {G_flicker.number_of_nodes()}')

Edges: 9114557
Nodes: 214626


In [ ]:
total_edges = G_flicker.number_of_edges()

In [ ]:
communities = nx.community.louvain_communities(G_flicker)

In [ ]:
sorted_communities_modularity = sort_communities_mod(G_flicker,communities)

In [ ]:
sorted_nodes_mod_degree = sort_nodes_degree(G_flicker,sorted_communities_modularity)

In [ ]:
G_flicker_mod_deg = nx.DiGraph()
G_flicker_mod_deg.add_nodes_from(sorted_nodes_mod_degree)
G_flicker_mod_deg.add_edges_from(G_flicker.edges)

In [ ]:
export_adj(G_flicker_mod_deg,'flicker_mod_deg')

flicker_mod_deg.graph-txt created


# Comments from the code review session

1. When plotting the graphs, you were wondering why the previous graphs were disepearing. Maybe it is because plt.close() is called at the end of the function. This closes the figure after saving it, ensuring that it doesn't stay in memory. This behavior is intentional and is used to free up resources in a script that generates multiple plots. However, it also means that the figure is no longer accessible for further inspection or interaction.

Code is readable.
It is a good idea to plot the gap distributions.
The part where you test the functions is well organized, but the part where you define them needs a little bit of cleaning (divide the code into different cells).  
More comments are needed to explain the code.
